# Lab3c - Image Classification

## Unsupervised classification

Unsupervised classification is a machine learning technique that groups similar pixels into classes based on their spectral characteristics. Earth Engine provides the `ee.Clusterer` class for performing unsupervised classification. The supported clustering algorithms include: `wekaKmeans`, `wekaXmeans`, `wekaCascadeKMeans`, `wekaCobweb`, and `wekaVQ`. The general workflow for performing unsupervised classification in Earth Engine is as follows:

1. Prepare a multiband image for classification.
2. Generate training samples from the image.
3. Initialize a clusterer and adjust the parameters as needed.
4. Train the clusterer using the training samples.
5. Apply the clusterer to the image.
6. Label the clusters as needed.
7. Export the classified image.

The following example demonstrates how to perform unsupervised classification on a Landsat 9 image. First, filter the Landsat 9 image collection by a region of interest and date range. Select the least cloudy image and select the seven spectral bands. Note that a scaling factor of 0.0000275 and a bias of -0.2 are applied to the image to convert the DN values to reflectance values:

In [ ]:
# %pip install geemap geedim

In [2]:
import ee
import geemap

c:\Users\C00553090\AppData\Local\miniconda3\envs\hypercoast\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [4]:
ee.Authenticate()
ee.Initialize()

In [5]:
m = geemap.Map()

point = ee.Geometry.Point([-90.7664, 29.9411])

image = (
    ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
    .filterBounds(point)
    .filterDate('2022-01-01', '2022-12-31')
    .sort('CLOUD_COVER')
    .first()
    .select('SR_B[1-7]')
)

region = image.geometry()
# Convert from integer to float
image = image.multiply(0.0000275).add(-0.2).set(image.toDictionary())
vis_params = {'min': 0, 'max': 0.3, 'bands': ['SR_B4', 'SR_B3', 'SR_B2']}

m.center_object(region, 8)
m.add_layer(image, vis_params, "Landsat-9")
m

Map(center=[30.302792321397934, -90.15508446571599], controls=(WidgetControl(options=['position', 'transparent…

Next, use the `get_info()` function to inspect the image metadata and inspect it in a tree structure:

In [6]:
geemap.get_info(image)

Tree(nodes=(Node(name='Image  (7 bands)', nodes=(Node(icon='file', name='type: Image'), Node(name='bands: List…

You can also get an image property using the `get()` method. For example, to get the image acquisition date:

In [7]:
image.get('DATE_ACQUIRED').getInfo()

'2022-04-07'

To check the image cloud cover:

In [8]:
image.get('CLOUD_COVER').getInfo()

0.01

With the image ready, you can now generate training samples. You can specify a region where the training samples will be generated. There are several ways to create a region for generating training samples:

- Draw a shape (e.g., rectangle) on the map and the use `region = m.user_roi`
- Define a geometry, such as `region = ee.Geometry.Rectangle([xmin, ymin, xmax, ymax])`
- Create a buffer around a point, such as `region = ee.Geometry.Point([x, y]).buffer(v)`
- If you don't define a region, it will use the image footprint by default

The following code generates 5000 training samples from the image and add them to the m. Note that the `region` parameter is not specified, so the image footprint will be used as the region:

In [9]:
training = image.sample(
    **{
        # "region": region,
        'scale': 30,
        'numPixels': 5000,#'numPixels': 5000: This limits the sample to 5,000 pixels from the image. It means that 5,000 random pixels will be extracted from the image.
        'seed': 0, # 'seed': 0: This provides a seed for randomization. The same seed ensures that the sampling process is reproducible; the same pixels will be selected every time you run this code.
        'geometries': True,  # Set this to False to ignore geometries
    }
)

m.add_layer(training, {}, 'Training samples') # 'Training samples' is the name of the layer that will be displayed on the map.
m

Map(bottom=27367.0, center=[29.863114194982682, -88.77906605090608], controls=(WidgetControl(options=['positio…

To inspect the attribute table of training data, use the `ee_to_df()` function on the first few features of the collection:

In [18]:
geemap.ee_to_df(training.limit(20))

,SR_B1,SR_B2,SR_B3,SR_B4,SR_B5,SR_B6,SR_B7
0,0.007653,0.018487,0.021650,0.016288,0.020742,0.033282,0.032512
1,0.037518,0.040625,0.070765,0.051818,0.329485,0.193003,0.106598
2,0.038040,0.042907,0.073873,0.057015,0.335865,0.187833,0.097577
3,0.037105,0.043182,0.062845,0.052725,0.047308,0.036307,0.030230
4,-0.007142,0.004242,0.017388,0.022172,0.039772,0.032018,0.027095
5,-0.001395,0.000667,0.001850,-0.004805,-0.001257,0.011475,0.012712
6,-0.003980,0.009632,0.015600,0.010293,0.016095,0.026600,0.026627
7,0.002812,0.009990,0.014198,0.010650,0.014967,0.026353,0.025968
8,-0.007747,0.002125,0.012987,0.004875,0.006717,0.018982,0.019285
9,0.040350,0.045410,0.053990,0.049177,0.045300,0.042742,0.037765


The training data is ready. Next, you need to initialize a clusterer and train it using the training data. The following code initializes a `wekaKmeans` clusterer and trains it using the training data by specifying the number of clusters (e.g., 5):

In [10]:
# This specifies the number of clusters you want to create.
# In this case, 5 clusters will be formed. These clusters represent different groups or classes of pixels that share similar characteristics based on their spectral values (or other features).
n_clusters = 5

#ee.Clusterer is a clustering module in Google Earth Engine that provides access to clustering algorithms.
# wekaKMeans() is the specific clustering algorithm being used, which is the K-Means algorithm from the WEKA library (a machine learning toolkit).
# K-Means is an unsupervised learning algorithm used to partition the data into K clusters. The goal is to group pixels based on their similarity, with the number of clusters defined by n_clusters (in this case, 5).
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

With the clusterer trained, you can now apply it to the image. The following code applies the clusterer to the image and adds the classified image to the map with a random color palette:

In [11]:
result = image.cluster(clusterer)
m.add_layer(result.randomVisualizer(), {}, 'clusters')
m

Map(bottom=7182.0, center=[27.644606381943326, -77.05810546875001], controls=(WidgetControl(options=['position…

Note the value and color of each cluster are randomly assigned. Use the Inspector tool to inspect the value of each cluster and label them as needed. Define a legend dictionary with pairs of cluster labels and colors, which can be used to create a legend for the classified image:

In [12]:
legend_dict = {
    'Developed': '#466b9f',
    'Forest': '#ab0000',
    'Crop': '#d99282',
    'Wetlands': '#1c5f2c',
    'Open Water': '#ab6c28'

}

palette = list(legend_dict.values())

m.add_layer(
    result, {'min': 0, 'max': 4, 'palette': palette}, 'Labelled clusters'
)
m.add_legend(title='Land Cover Type',legend_dict=legend_dict , position='bottomright')
m

Map(bottom=54330.0, center=[30.11186984923527, -89.49737548828126], controls=(WidgetControl(options=['position…

The unsupervised classification result is shown in {numref}`ch06_unsupervised_classification`.

Finally, you can export the classified image to your computer. Specify the image region, scale, and output file path as needed:

In [ ]:
geemap.download_ee_image(image, filename='unsupervised.tif', region=region, scale=90)

## Supervised classification

Supervised classification is a machine learning technique that uses labeled training data to classify an image. The training data is used to train a classifier, which is then applied to the image to generate a classified image. Earth Engine provides the `ee.Classifier` class for performing supervised classification. The supported supervised classification algorithms include: [Classification and Regression Trees (CART)](https://en.wikipedia.org/wiki/Decision_tree_learning), [Support Vector Machine (SVM)](https://en.wikipedia.org/wiki/Support_vector_machine), [Random Forest](https://en.wikipedia.org/wiki/Random_forest), [Naive Bayes](https://en.wikipedia.org/wiki/Naive_Bayes_classifier), and [Gradient Tree Boost](https://en.wikipedia.org/wiki/Gradient_boosting). The general workflow for supervised classification is as follows:

1. Prepare an image for classification.
2. Collect training data. Each training sample should have a class label and a set of properties storing numeric values for the predictors.
3. Initialize a classifier and set its parameters as needed.
4. Train the classifier using the training data.
5. Apply the classifier to the image.
6. Perform accuracy assessment.
7. Export the classified image.

In this section, you will learn how to perform supervised classification using the CART algorithm. You can easily adapt the code to other supervised classification algorithms, such as SVM and Random Forest. We will use labeled training data from the [USGS National Land Cover Database (NLCD)](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD_RELEASES_2019_REL_NLCD) dataset and train a CART classifier using the training data. The trained classifier will then be applied to the Landsat-9 image to generate a classified image.

First, filter the [Landsat 8 image collection](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2) to select a cloud-free image acquired in 2019 for your region of interest:

In [ ]:
m = geemap.Map()
point = ee.Geometry.Point([-122.4439, 37.7538])

image = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterBounds(point)
    .filterDate('2019-01-01', '2020-01-01')
    .sort('CLOUD_COVER')
    .first()
    .select('SR_B[1-7]')
)

image = image.multiply(0.0000275).add(-0.2).set(image.toDictionary())
vis_params = {'min': 0, 'max': 0.3, 'bands': ['SR_B5', 'SR_B4', 'SR_B3']}

m.center_object(point, 8)
m.add_layer(image, vis_params, "Landsat-8")
m

Use the `get_info()` function to check the image properties:

In [ ]:
geemap.get_info(image)

To get a specific image property, use the `get()` method with the property name as the argument. For example, to retrieve the image acquisition date:

In [ ]:
image.get('DATE_ACQUIRED').getInfo()

To check the cloud cover of the image:

In [ ]:
image.get('CLOUD_COVER').getInfo()

Next, create a training dataset from the NLCD dataset, which is a 30-m resolution dataset covering the conterminous United States. The NLCD dataset contains 21 land cover classes. A detailed description of each NLCD land cover type can be found at <https://bit.ly/3EyvacV>. The following code filters the NLCD dataset to select the land cover image of interest and clips the dataset to the region of interest (the footprint of the selected Landsat image in the previous step):

In [ ]:
nlcd = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2019')
landcover = nlcd.select('landcover').clip(image.geometry())
m.add_layer(landcover, {}, 'NLCD Landcover')
m

With the land cover image ready, we can now sample the image to collect training data with land cover labels. Similar to the unsupervised classification introduced above, you can specify a region of interest, scale, and number of points to sample. The following code samples 5000 points from the land cover image:

In [ ]:
points = landcover.sample(
    **{
        'region': image.geometry(),
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,
    }
)

m.add_layer(points, {}, 'training', False)

Note that the resulting number of training samples may be less than the specified number of points. This is because the sampling algorithm will discard pixels with no data values. To check the number of training samples:

In [ ]:
print(points.size().getInfo())

Revise the number of points to sample in the previous step if needed.

Next, we will add the spectral bands of the Landsat image to the training data. Note that the training data created from the previous step already contains the land cover labels (i.e., the `landcover` property). The following code adds the seven spectral bands of the Landsat image to the training data:

In [ ]:
bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
label = 'landcover'
features = image.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 30}
)

Dislay the attribute table of the training data using the `ee_to_df()` function:

In [ ]:
geemap.ee_to_df(features.limit(5))

The training dataset is ready. You can now train a classifier using the training data. The following code initializes a [CART classifier](https://developers.google.com/earth-engine/apidocs/ee-classifier-smilecart) and trains it using the training data:

In [ ]:
params = {

    'features': features,
    'classProperty': label,
    'inputProperties': bands,

}
classifier = ee.Classifier.smileCart(maxNodes=None).train(**params)

The `features` parameter specifies the training data. The `classProperty` parameter specifies the property name of the training data that contains the class labels. The `inputProperties` parameter specifies the property names of the training data that contain the predictor values.

All Earth Engine classifiers have a `train()` function to train the classifier using the training data. The CART classifier has a `maxNodes` parameter to specify the maximum number of nodes in the tree. The default value is `None`, which means that the tree will be grown until all leaves are pure or until all leaves contain less than 5 training samples.

Since the classifier has been trained, you can now apply it to the Landsat image to generate a classified image. Make sure you use the same spectral bands as the training data. The following code applies the trained classifier to the selected Landsat image and adds the classified image with a random color palette to the map:

In [ ]:
classified = image.select(bands).classify(classifier).rename('landcover')
m.add_layer(classified.randomVisualizer(), {}, 'Classified')
m

To compare the classified image with the referenced NLCD land cover image, it is better to use the same color palette. To set the color palette of an Earth Engine image with a predefined palette, set the `bandname_class_values` and `bandname_class_palette` properties of the image. For example, the NLCD land cover has the `landcover_class_values` and `landcover_class_palette` properties. When the land cover band is added to the map, the color palette will be automatically applied so that users don't have to specify the color palette manually. To check the color palette of the NLCD land cover image:

In [ ]:
geemap.get_info(nlcd)

We can use the same approach to set the color palette of the classified image. Note that in the previous step, we already renamed the classified image band to `landcover`. Therefore, we can use the `landcover_class_values` and `landcover_class_palette` properties to set the color palette of the classified image. The following code sets the color palette of the classified image:

In [ ]:
class_values = nlcd.get('landcover_class_values')
class_palette = nlcd.get('landcover_class_palette')
classified = classified.set({
    'landcover_class_values': class_values,
    'landcover_class_palette': class_palette
})

The classified image should now have the same color palette as the NLCD land cover image. Add the classified image and associated legend to the map:

In [ ]:
m.add_layer(classified, {}, 'Land cover')
m.add_legend(title="Land cover type", builtin_legend='NLCD')
m

Use the layer control widget to change the opacity of the classified image and compare it visually with the NLCD land cover image. You might need to use the full-screen mode as the legend may block the view of layer control widget.

Finally, you can export the classified image to your computer:

In [ ]:
geemap.download_ee_image(
    landcover,
    filename='supervised.tif',
    region=image.geometry(),
    scale=30
    )

## Accuracy assessment

After performing image classification, you may want to assess the accuracy of the classification. Earth Engine provides several functions for assessing the accuracy of a classification. In this section, we will classify a Sentinel-2 image using [random forest](https://en.wikipedia.org/wiki/Random_forest) and assess the accuracy of the classification.

First, filter the Sentinel-2 image collection and select an image for your region of interest:

In [ ]:
m = geemap.Map()
point = ee.Geometry.Point([-122.4439, 37.7538])

img = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterBounds(point)
    .filterDate('2020-01-01', '2021-01-01')
    .sort('CLOUDY_PIXEL_PERCENTAGE')
    .first()
    .select('B.*')
)

vis_params = {'min': 100, 'max': 3500, 'bands': ['B11',  'B8',  'B3']}

m.center_object(point, 9)
m.add_layer(img, vis_params, "Sentinel-2")
m

The [ESA 10-m WorldCover](https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v100) can be used to create labeled training data. First, we need to remap the land cover class values to a 0-based sequential series so that we can create a confusion matrix later:

In [ ]:
lc = ee.Image('ESA/WorldCover/v100/2020')
classValues = [10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 100]
remapValues = ee.List.sequence(0, 10)
label = 'lc'
lc = lc.remap(classValues, remapValues).rename(label).toByte()

Next, add the ESA land cover as a band of the Sentinel-2 reflectance image and sample 100 pixels at a 10m scale from each land cover class within the region of interest:

In [ ]:
sample = img.addBands(lc).stratifiedSample(**{
  'numPoints': 100,
  'classBand': label,
  'region': img.geometry(),
  'scale': 10,
  'geometries': True
})

Add a random value field to the sample and use it to approximately split 80% of the features into a training set and 20% into a validation set:

In [ ]:
sample = sample.randomColumn()
trainingSample = sample.filter('random <= 0.8')
validationSample = sample.filter('random > 0.8')

With the training data ready, we can train a random forest classifier using the training data.

The following code trains a random forest classifier with 10 trees:

In [ ]:
trainedClassifier = ee.Classifier.smileRandomForest(numberOfTrees=10).train(**{
  'features': trainingSample,
  'classProperty': label,
  'inputProperties': img.bandNames()
})

To get information about the trained classifier:

In [ ]:
print('Results of trained classifier', trainedClassifier.explain().getInfo())

To get a confusion matrix and overall accuracy for the training sample:

In [ ]:
trainAccuracy = trainedClassifier.confusionMatrix()
trainAccuracy.getInfo()

```text
[[81, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 83, 1, 0, 0, 0, 0, 0, 0],
 [1, 1, 73, 2, 0, 0, 0, 0, 0],
 [0, 0, 1, 77, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 81, 1, 0, 0, 0],
 [0, 1, 2, 3, 2, 70, 0, 0, 2],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 71, 0],
 [1, 0, 0, 1, 0, 4, 0, 0, 71]]
```

The horizontal axis of the confusion matrix corresponds to the input classes, and the vertical axis corresponds to the output classes. The rows and columns start at class 0 and increase sequentially up to the maximum class value, so some rows or columns might be empty if the input classes aren't 0-based or sequential. That's the reason why we remapped the ESA land cover class values to a 0-based sequential series earlier. Note that your confusion matrix may look slightly different from the one shown above as the training data is randomly sampled.

The overall accuracy essentially tells us what proportion of al the reference sites was mapped correctly. The overall accuracy is usually expressed as a percent, with 100% accuracy being a perfect classification where all reference sites were classified correctly.

In [ ]:
trainAccuracy.accuracy().getInfo()

The Kappa Coefficient is generated from a statistical test to evaluate the accuracy of a classification. Kappa essentially evaluates how well the classification performed as compared to just randomly assigning values, i.e., did the classification do better than random? The Kappa Coefficient can range from -1 to 1. A value of 0 indicated that the classification is no better than a random classification. A negative number indicates the classification is significantly worse than random. A value close to 1 indicates that the classification is significantly better than random.

In [ ]:
trainAccuracy.kappa().getInfo()

To get a confusion matrix and overall accuracy for the validation sample:

In [ ]:
validationSample = validationSample.classify(trainedClassifier)
validationAccuracy = validationSample.errorMatrix(label, 'classification')
validationAccuracy.getInfo()

```text
[[13, 1, 3, 0, 1, 0, 0, 0, 0],
 [0, 11, 2, 0, 1, 0, 0, 0, 2],
 [1, 0, 12, 7, 1, 2, 0, 0, 0],
 [0, 3, 6, 9, 3, 1, 0, 0, 0],
 [2, 0, 3, 0, 10, 2, 0, 0, 0],
 [1, 1, 1, 3, 7, 6, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 29, 0],
 [2, 2, 2, 0, 2, 2, 0, 0, 13]]
```

To compute the overall accuracy for the validation sample:

In [ ]:
validationAccuracy.accuracy().getInfo()

Producer's Accuracy, also known as the Sensitivity or Recall, is a measure of how well a classifier correctly identifies positive instances. It is the ratio of true positive classifications to the total number of actual positive instances. This metric is used to evaluate the performance of a classifier when the focus is on minimizing false negatives (i.e., instances that are actually positive but are classified as negative).

In [ ]:
validationAccuracy.producersAccuracy().getInfo()

On the other hand, Consumer's Accuracy, also known as Precision, is a measure of how well a classifier correctly identifies negative instances. It is the ratio of true negative classifications to the total number of actual negative instances. This metric is used to evaluate the performance of a classifier when the focus is on minimizing false positives (i.e., instances that are actually negative but are classified as positive).

In [ ]:
validationAccuracy.consumersAccuracy().getInfo()

The confusion matrices can be saved as a csv file:

In [ ]:
import csv

with open("training.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(trainAccuracy.getInfo())

with open("validation.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(validationAccuracy.getInfo())

If the validation accuracy is acceptable, the trained classifier can then be applied to the entire image:

In [ ]:
imgClassified = img.classify(trainedClassifier)

Lastly, add the resulting data layers (e.g., Sentinel-2 image, classified image, training and validation samples) to the m. Use the layer control widget the change the opacity of the classified image and compare it to the ESA WorldCover.

In [ ]:
classVis = {
  'min': 0,
  'max': 10,
  'palette': ['006400' ,'ffbb22', 'ffff4c', 'f096ff', 'fa0000', 'b4b4b4',
            'f0f0f0', '0064c8', '0096a0', '00cf75', 'fae6a0']
}
m.add_layer(lc, classVis, 'ESA Land Cover', False)
m.add_layer(imgClassified, classVis, 'Classified')
m.add_layer(trainingSample, {'color': 'black'}, 'Training sample')
m.add_layer(validationSample, {'color': 'white'}, 'Validation sample')
m.add_legend(title='Land Cover Type', builtin_legend='ESA_WorldCover')
m.center_object(img)
m